In [1]:
#importing required libraries for the program to run
!pip install matplotlib seaborn
!pip install scikit-learn tensorflow seaborn imbalanced-learn


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\ahada\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\ahada\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
!pip install matplotlib seaborn scikit-learn tensorflow imbalanced-learn


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\ahada\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [1]:
#importing required libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
df = pd.read_csv('data/creditcard.csv')

In [3]:
#checking How much data we have, details about columsn and any missing data
df.shape
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [4]:
#gives us the details about fruad vs legitimate values, in this case we have 0 is legitimate and 1 shows us the fraud values
df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [4]:
#visualization of the data 
sns.countplot(x='Class', data=df)
plt.title('Class Distribution')
plt.show()

NameError: name 'sns' is not defined